<a href="https://colab.research.google.com/github/rahulzach/BeatBot-Music-Genre-and-Singer-Classification-Website/blob/main/ArtistRecogFinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import os
import pickle
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import os
import pickle
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# Function to extract MFCC features from audio data
def extract_features(audio_data, sample_rate=22050, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

In [17]:
# Function to load audio data and extract features
def load_audio_data(directory):
    X = []
    y = []

    for speaker_dir in os.listdir(directory):
        speaker_path = os.path.join(directory, speaker_dir)
        if os.path.isdir(speaker_path):
            for audio_file in os.listdir(speaker_path):
                audio_path = os.path.join(speaker_path, audio_file)
                audio_data, sample_rate = librosa.load(audio_path, sr=None)
                features = extract_features(audio_data, sample_rate)
                X.append(features)
                y.append(speaker_dir)

    return np.array(X), np.array(y)

In [18]:
# Load audio data
data_directory = "/content/drive/MyDrive/Colab Notebooks/DatasetSpeaker/speeches"
X, y = load_audio_data(data_directory)

In [19]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [20]:
# Train the model with regularization
model_regularized = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)
model_regularized.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [21]:
# Evaluate the regularized model on training, validation, and test sets
train_accuracy_regularized = accuracy_score(y_train, model_regularized.predict(X_train))
valid_accuracy_regularized = accuracy_score(y_valid, model_regularized.predict(X_valid))
test_accuracy_regularized = accuracy_score(y_test, model_regularized.predict(X_test))

print("Training accuracy (regularized):", train_accuracy_regularized)
print("Validation accuracy (regularized):", valid_accuracy_regularized)
print("Test accuracy (regularized):", test_accuracy_regularized)

Training accuracy (regularized): 0.9964497041420118
Validation accuracy (regularized): 0.9245283018867925
Test accuracy (regularized): 0.9150943396226415


In [22]:
# Save the model to a pickle file
model_filename = "speaker_identification_model.pkl"
with open(model_filename, "wb") as f:
    pickle.dump(model_regularized, f)